<a href="https://colab.research.google.com/github/Otter2022/Oil_Marketing_Campaign/blob/main/tokens10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
#tokens10, first 3 tokens are handled only
import csv
import re
import sys
import os

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

source_data = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Data/'
source_data_output = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Data_Output/'
source_code = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Code/'
admin = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Admin/'
backups = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Backups/'

#The lists contained in the marketing_compaign_oil.py file serves as a header for name pre-processing
#contains exclude, compiled_patterns for exclude, fix_abbreviations, honorifics, suffixes, suffixes_roman, etc.
marketing_campaign_oil = source_code+'marketing_campaign_oil.py'
with open(marketing_campaign_oil, 'r') as f:
    exec(f.read())
    f.close()

input_file = source_data + 'tokens10_1_CSV.csv'
#input_file = basepath + 'tokens2_test_csv.csv'
output_file = source_data_output + 'tokens10_formatted_output.csv'

#test the reading of the marketing_campaign_oil package
print(honorifics)
print(suffixes)
print(suffixes_roman)
print(fix_abbreviations)

Mounted at /content/drive
['Dr', 'Doctor', 'Mr', 'Mrs', 'Ms', 'Miss', 'Prof', 'Professor', 'Sir', 'Madam', 'Dame', 'Lord', 'Lady', 'Rev', 'Reverend', 'Hon', 'Honorable', 'Capt', 'Captain', 'Major', 'Col', 'Colonel', 'Gen', 'General', 'Adm', 'Admiral', 'Sgt', 'Sergeant', 'Lt', 'Lieutenant', 'Chief', 'Officer', 'Judge', 'Magistrate', 'Sen', 'Senator', 'Rep', 'Representative', 'Amb', 'Ambassador', 'Pres', 'President', 'VP', 'Vice President', 'Gov', 'Governor', 'Mayor']
['Jr', 'Sr', 'Esq', 'Esquire', 'PhD', 'MD', 'DDS', 'DVM', 'JD', 'CPA', 'RN', 'LPN', 'PE', 'Ret']
['V', 'IV', 'III', 'II', 'I']
{'FAM': 'Family', 'TR': 'Trust', 'REV': 'Revocable'}


In [ ]:
def is_excluded(input_string):
    # Check if any compiled pattern matches the input string
    #I want to print the input_strings that are excluded
    for pattern in compiled_patterns:
        if pattern.match(input_string):
            print(f"Excluded: {input_string}")
            return True
    return False

def abbreviate(token):
  if token in fix_abbreviations.keys():
    return fix_abbreviations[token]
  else:
    return token


def rearrange_tokens(name):
    # Split the name into tokens
    tokens = name.split()
    print(tokens)
    # Process each token to fix abbreviations
    tokens = [abbreviate(token.strip()) for token in tokens]
    clean_tokens = [token.replace('\ufeff', '') for token in tokens]

    tokens = [token.lower() for token in clean_tokens]
    # Capitalize tokens that are not in honorifics
    tokens = [token.capitalize() if token.lower() not in honorifics else token for token in tokens]
    # Capitalize tokens that are not in suffixes
    tokens = [token.capitalize() if token.lower() not in suffixes and token.lower() not in suffixes_roman else token for token in tokens]

    if len(tokens) == 10:

        token1, token2, token3, token4, token5, token6, token7, token8, token9, token10 = tokens[0], tokens[1], tokens[2], tokens[3], tokens[4], tokens[5], tokens[6], tokens[7], tokens[8], tokens[9]

        print(f"Name: {name}")
        # Exclude if any token for the owner name is in the exclude list
        for token in tokens:
          if is_excluded(token):
              print(f"Excluded: {name}")
              return "Miss"

        # Rearrange the tokens based on the rules
        if token1 in honorifics:
            return f"{token1} {token3} {token2} {token4} {token5} {token6} {token7} {token8} {token9} {token10}"
        elif token3 in suffixes or token3 in suffixes_roman:
            return f"{token1} {token2} {token3} {token4} {token5} {token6} {token7} {token8} {token9} {token10}"
        elif token2 == "Trust":
            return f"{token1} {token3} {token2} {token4} {token5} {token6} {token7} {token8} {token9} {token10}"
        elif token3 == "Trust":#do nothing
            return f"{token1} {token2} {token3} {token4} {token5} {token6} {token7} {token8} {token9} {token10}"
        elif token2 == "Family":
            return f"{token1} {token3} {token2} {token4}{token5} {token6} {token7} {token8} {token9} {token10}"
        elif token3 == "Family":
            return f"{token1} {token3} {token2} {token4} {token5} {token6} {token7} {token8} {token9} {token10}"
        elif token1.isalpha() and token2.isalpha() and token3.isalpha():
            return f"{token2} {token3} {token1} {token4} {token5} {token6} {token7} {token8} {token9} {token10}"
        else: #Smith A J, only rearrange first 3 tokens
            return f"{token2} {token3} {token1} {token4} {token5} {token6} {token7} {token8} {token9} {token10}"
    return "Miss"


def process_names(input_file, output_file):
    with open(input_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        names = [row[0] for row in reader]

    names = [name.strip() for name in names]

    formatted_names = [(name, rearrange_tokens(name)) for name in names]
    misses = 0
    total = len(formatted_names)
    print(f"Total: {total}")

    try:
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for original, rearranged in formatted_names:
                if rearranged == 'Miss':
                    misses += 1
                else:
                    writer.writerow([original, rearranged])
    except TypeError:
        with open(output_file, 'w') as csvfile:
            writer = csv.writer(csvfile)
            for original, rearranged in formatted_names:
                if rearranged == 'Miss':
                    misses += 1
                else:
                    writer.writerow([original, rearranged])

    return misses, total



In [ ]:
def main():
    misses,total = process_names(input_file, output_file)
    print(f"Miss percentage {(float(misses)/float(total))*100.00}%.")

if __name__ == "__main__":
    main()

['\ufeffPENN', 'D', 'LEE', 'TRUST', 'A', 'P', 'D', 'L', 'TR', 'B']
Name: ﻿PENN D LEE TRUST A P D L TR B
Total: 1
Miss percentage 0.0%.
